In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
	# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()


In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [5]:
#load in a sql function to use columns
from pyspark.sql.functions import col

#filter total_votes
filtered_votes_df = vine_df.filter(col('total_votes') > 20)
filtered_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|     

In [6]:
#retrieve all rows where helpful_votes/total_votes >= 50%
divided_votes_df = filtered_votes_df.filter(col('helpful_votes')/col('total_votes') >= 0.5)
divided_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|                N|
|R1RKEBF47UEIVH|          5|           26|         30|   N|     

In [21]:
#retrieve rows where review was written as part of vine program
vine_reviews_df = divided_votes_df.filter(col('vine') == 'Y')
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|
|R3NTGOCUZB33JK|          5|           40|         47|   Y|                Y|
| RQX3RHGMG7UBN|          5|          186|        200|   Y|                N|
|R3E6QYKZDST042|          4|           33|         39|   Y|                N|
|R2GR106PWUN85O|          4|           36|         39|   Y|     

616

In [23]:
#number of vine reviews
vine_reviews_df.count()

616

In [8]:
#retrieve rows where review was NOT written as part of vine program
non_vine_reviews_df = divided_votes_df.filter(col('vine') == 'N')
non_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|                N|
|R1RKEBF47UEIVH|          5|           26|         30|   N|     

In [22]:
#number of non vine reviews
on_vine_reviews_df.count()

69432

In [10]:
#total number of reviews
vine_df.count()

5115666

In [11]:
#number of 5 star reviews
vine_df.filter(col('star_rating') == 5).count()

3254753

In [24]:
#number of 5 star vine reviews
vine_df.filter((col('star_rating') == 5) & (col('vine') == 'Y')).count()

12429

In [25]:
#number of 5 star non vine reviews
vine_df.filter((col('star_rating') == 5) & (col('vine') == 'N')).count()

3242324

In [17]:
#percentage of 5 star reviews for paid 
(vine_df.filter((col('star_rating') == 5) & (col('vine') == 'Y')).count() / vine_df.filter(col('star_rating') == 5).count()) * 100

0.38187229568572484

In [18]:
#percentage of 5 star reviews for non paid 
(vine_df.filter((col('star_rating') == 5) & (col('vine') == 'N')).count() / vine_df.filter(col('star_rating') == 5).count()) * 100

99.61812770431428